In [1]:
import pandas as pd
from math import comb
from ax.service.ax_client import AxClient
import time
import sys

sys.path.append('../../../0_helper_functions')
import sdlnano as sdl
import calculation as calc

In [2]:
# Design space size calculation


def calculate_combinations(num_variables, total_sum, step):
    steps = int(total_sum / step)
    return comb(steps + num_variables - 1, num_variables - 1)

# Define parameters
step = 0.05
total_sum = 1

# Calculate combinations for each group
count_group1 = calculate_combinations(10, total_sum, step)  # 3 Solid lipids + 3 Polymers + 3 Liquid lipids + 1 Drug
count_group2 = calculate_combinations(4, total_sum, step)   # 3 Surfactants + 1 Water

# Calculate total combinations
total_combinations = count_group1 * count_group2


print("organic design space size: ", round(count_group1/(1e6),2), 'Million')
print("aqueous design space size: ", round(count_group2/(1e3),2), 'Thousand')
print("design space size: ", round(total_combinations/(1e9),2), 'Billion')


organic design space size:  10.02 Million
aqueous design space size:  1.77 Thousand
design space size:  17.74 Billion


# Check the iteration number

In [3]:
iteration = sdl.get_iteration_number()
print("This is the iteration: #", iteration)

This is the iteration: # 3


# Check the drug

In [4]:
drug = 'ACE'
print("This is the drug:", drug)

This is the drug: ACE


# Optimizer initialization (N/A)

# Generate recommendations

In [5]:
time_start = time.time()

previous_optimizer_file_name = '../iteration_' + str(iteration-1) + '/optimizer/optimizer_load_' + str(iteration-1) + '.json'
# generate recomms
ax_client_init = AxClient.load_from_json_file(previous_optimizer_file_name)
unlabeled_trial, ax_client_design = sdl.generate_trials(ax_client=ax_client_init, num_of_trials = 16, drug = drug, bopt=1)

time_end = time.time()

print(round((time_end-time_start)/60), 'min')

/Users/zeqing/opt/anaconda3/envs/sdlnano_plot/lib/python3.11/site-packages/botorch/acquisition/multi_objective/logei.py:418: RuntimeWarning: `cache_root` is only supported for GPyTorchModels that are not MultiTask models and don't produce a TransformedPosterior. Got a model of type <class 'botorch.models.model.ModelList'>. Setting `cache_root = False`.
  NoisyExpectedHypervolumeMixin.__init__(
[INFO 08-02 17:48:53] ax.service.ax_client: Generated new trial 48 with parameters {'Drug': 53, 'SL_1': 0, 'SL_2': 4, 'SL_3': 0, 'LL_1': 0, 'LL_2': 0, 'LL_3': 0, 'P_1': 0, 'P_2': 0, 'P_3': 0, 'S_1': 0, 'S_2': 99, 'S_3': 0, 'Water': 0} using model BoTorch.
/Users/zeqing/opt/anaconda3/envs/sdlnano_plot/lib/python3.11/site-packages/ax/core/data.py:293: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the rel

20 min


In [6]:
# process the trails into ratios

unlabeled_trial_processed = sdl.process_trails(unlabeled_trial)
unlabeled_trial_processed.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.000000,16.000000,16.0,16.0,16.000000,16.0,16.000000,16.0,16.000000,16.0,16.0,16.0,16.0
mean,55.500000,0.870375,0.022187,0.004375,0.0,0.0,0.011562,0.0,0.033813,0.0,0.057688,0.0,1.0,0.0,0.0
std,4.760952,0.263428,0.088750,0.017500,0.0,0.0,0.046250,0.0,0.135250,0.0,0.230750,0.0,0.0,0.0,0.0
min,48.000000,0.077000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,1.0,0.0,0.0
25%,51.750000,0.901250,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,1.0,0.0,0.0
50%,55.500000,1.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,1.0,0.0,0.0
75%,59.250000,1.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,1.0,0.0,0.0
max,63.000000,1.000000,0.355000,0.070000,0.0,0.0,0.185000,0.0,0.541000,0.0,0.923000,0.0,1.0,0.0,0.0


In [7]:
unlabeled_trial
unlabeled_trial.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.0000,16.00,16.0,16.0,16.00,16.0,16.00,16.0,16.00,16.0,16.0000,16.0,16.0
mean,55.500000,56.875000,2.4375,0.25,0.0,0.0,0.75,0.0,3.75,0.0,2.25,0.0,99.9375,0.0,0.0
std,4.760952,33.741913,9.7500,1.00,0.0,0.0,3.00,0.0,15.00,0.0,9.00,0.0,0.2500,0.0,0.0
min,48.000000,1.000000,0.0000,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,99.0000,0.0,0.0
25%,51.750000,51.000000,0.0000,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,100.0000,0.0,0.0
50%,55.500000,53.000000,0.0000,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,100.0000,0.0,0.0
75%,59.250000,78.250000,0.0000,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,100.0000,0.0,0.0
max,63.000000,100.000000,39.0000,4.00,0.0,0.0,12.00,0.0,60.00,0.0,36.00,0.0,100.0000,0.0,0.0


In [8]:
# Convert ratios to volumes

transfer = calc.converter(unlabeled_trial_processed)
transfer.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Solvent
count,16.000000,16.000000,16.000000,16.00000,16.0,16.0,16.000000,16.0,16.000000,16.0,16.000000,16.0,16.0,16.0,16.0,16.0
mean,55.500000,304.631250,7.765625,1.53125,0.0,0.0,4.046875,0.0,11.834375,0.0,20.190625,0.0,1000.0,0.0,0.0,0.0
std,4.760952,92.199937,31.062500,6.12500,0.0,0.0,16.187500,0.0,47.337500,0.0,80.762500,0.0,0.0,0.0,0.0,0.0
min,48.000000,26.950000,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,1000.0,0.0,0.0,0.0
25%,51.750000,315.437500,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,1000.0,0.0,0.0,0.0
50%,55.500000,350.000000,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,1000.0,0.0,0.0,0.0
75%,59.250000,350.000000,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,1000.0,0.0,0.0,0.0
max,63.000000,350.000000,124.250000,24.50000,0.0,0.0,64.750000,0.0,189.350000,0.0,323.050000,0.0,1000.0,0.0,0.0,0.0


In [9]:
#dead_volume = 3.0 # mL

round(transfer[['Drug','SL_1', 'SL_2', 'SL_3', 'LL_1', 'LL_2', 'LL_3', 'P_1', 'P_2', 'P_3', 'S_1', 'S_2', 'S_3', 'Water', 'Solvent']].sum()/1000,1)

Drug        4.9
SL_1        0.1
SL_2        0.0
SL_3        0.0
LL_1        0.0
LL_2        0.1
LL_3        0.0
P_1         0.2
P_2         0.0
P_3         0.3
S_1         0.0
S_2        16.0
S_3         0.0
Water       0.0
Solvent     0.0
dtype: float64

In [10]:
# file path
ax_client_design_path = "optimizer/optimizer_design_" + str(iteration) + ".json"

unlabeled_trial_path = "data/unlabeled_" + str(iteration) + ".xlsx"

unlabeled_trial_processed_path = "data/unlabeled_processed_" + str(iteration) + ".xlsx"

transfer_path = 'data/transfer_' + str(iteration) + '.xlsx'


In [11]:
ax_client_design.save_to_json_file(ax_client_design_path)

unlabeled_trial.to_excel(unlabeled_trial_path, index=False)

unlabeled_trial_processed.to_excel(unlabeled_trial_processed_path, index=False)

transfer.to_excel(transfer_path, index = False)

# Generate new protocol

In [12]:
# Paths
template_protocol = '../../templates/OT_protocol_template.py'

output_script_path_1 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_1.py'
output_script_path_2 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_2.py'



In [13]:
# Update the script with Excel data
updated_script = calc.update_transfer_script(template_protocol, transfer_path, output_script_path_1, output_script_path_2) 


In [14]:
updated_script

('protocol/iteration_3_OT_2_protocol_1.py',
 'protocol/iteration_3_OT_2_protocol_2.py')

# Perform experiment

# Results

In [15]:
results = calc.parameters_and_complexity(iteration)
results

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Complexity,Complexity_STD
0,48,0.930,0.000,0.07,0,0,0.000,0,0.000,0,0.000,0,1,0,0,2,0
1,49,1.000,0.000,0.00,0,0,0.000,0,0.000,0,0.000,0,1,0,0,1,0
2,50,1.000,0.000,0.00,0,0,0.000,0,0.000,0,0.000,0,1,0,0,1,0
3,51,1.000,0.000,0.00,0,0,0.000,0,0.000,0,0.000,0,1,0,0,1,0
4,52,1.000,0.000,0.00,0,0,0.000,0,0.000,0,0.000,0,1,0,0,1,0
5,53,1.000,0.000,0.00,0,0,0.000,0,0.000,0,0.000,0,1,0,0,1,0
6,54,0.815,0.000,0.00,0,0,0.185,0,0.000,0,0.000,0,1,0,0,2,0
7,55,0.459,0.000,0.00,0,0,0.000,0,0.541,0,0.000,0,1,0,0,2,0
8,56,1.000,0.000,0.00,0,0,0.000,0,0.000,0,0.000,0,1,0,0,1,0
9,57,0.645,0.355,0.00,0,0,0.000,0,0.000,0,0.000,0,1,0,0,2,0


In [18]:
size_raw = calc.size_raw(iteration)
size_raw

,Data Quality,Item,Size,PD Index
0,Caution,A1,19.270895,5.707963e-01
1,Caution,A2,46.582253,5.707963e-01
2,Caution,A3,68.017543,5.707963e-01
3,Good,A4,25.191242,1.195982e-01
4,Good,A5,24.532312,1.017086e-01
5,Good,A6,24.070679,1.617709e-01
6,Good,B1,24.316344,1.627621e-01
7,Caution,B2,23.767897,3.515772e-01
8,Caution,B3,25.368287,4.332631e-01
9,Good,B4,24.373871,9.840926e-02


In [19]:
size_processed = calc.process_formulations(size_raw)
size_processed

,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,Formulation 1,1000.000000,0.000000,1.000000,0.000000,0
1,Formulation 2,24.598078,0.563169,0.127693,0.030838,1
2,Formulation 3,1000.000000,0.000000,1.000000,0.000000,0
3,Formulation 4,25.055996,0.964670,0.120510,0.031255,1
4,Formulation 5,24.776899,0.166927,0.149819,0.051115,1
5,Formulation 6,24.087662,0.153833,0.099604,0.064277,1
6,Formulation 7,198.047478,3.789760,0.380302,0.045840,1
7,Formulation 8,24.932233,1.245743,0.351534,0.091538,1
8,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0
9,Formulation 10,1000.000000,0.000000,1.000000,0.000000,0


In [20]:
results_final = pd.concat([results, size_processed], axis=1)
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,S_3,Water,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,48,0.930,0.000,0.07,0,0,0.000,0,0.000,0,...,0,0,2,0,Formulation 1,1000.000000,0.000000,1.000000,0.000000,0
1,49,1.000,0.000,0.00,0,0,0.000,0,0.000,0,...,0,0,1,0,Formulation 2,24.598078,0.563169,0.127693,0.030838,1
2,50,1.000,0.000,0.00,0,0,0.000,0,0.000,0,...,0,0,1,0,Formulation 3,1000.000000,0.000000,1.000000,0.000000,0
3,51,1.000,0.000,0.00,0,0,0.000,0,0.000,0,...,0,0,1,0,Formulation 4,25.055996,0.964670,0.120510,0.031255,1
4,52,1.000,0.000,0.00,0,0,0.000,0,0.000,0,...,0,0,1,0,Formulation 5,24.776899,0.166927,0.149819,0.051115,1
5,53,1.000,0.000,0.00,0,0,0.000,0,0.000,0,...,0,0,1,0,Formulation 6,24.087662,0.153833,0.099604,0.064277,1
6,54,0.815,0.000,0.00,0,0,0.185,0,0.000,0,...,0,0,2,0,Formulation 7,198.047478,3.789760,0.380302,0.045840,1
7,55,0.459,0.000,0.00,0,0,0.000,0,0.541,0,...,0,0,2,0,Formulation 8,24.932233,1.245743,0.351534,0.091538,1
8,56,1.000,0.000,0.00,0,0,0.000,0,0.000,0,...,0,0,1,0,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0
9,57,0.645,0.355,0.00,0,0,0.000,0,0.000,0,...,0,0,2,0,Formulation 10,1000.000000,0.000000,1.000000,0.000000,0


In [21]:
# µg/mL
results_final['Solu'] = results_final['Drug'] * 600 / 0.3
results_final['Solu_STD'] = 0

results_final.loc[results_final['Formulation Quality'] == 0, 'Solu'] = 0
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,48,0.930,0.000,0.07,0,0,0.000,0,0.000,0,...,2,0,Formulation 1,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
1,49,1.000,0.000,0.00,0,0,0.000,0,0.000,0,...,1,0,Formulation 2,24.598078,0.563169,0.127693,0.030838,1,2000.0,0
2,50,1.000,0.000,0.00,0,0,0.000,0,0.000,0,...,1,0,Formulation 3,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
3,51,1.000,0.000,0.00,0,0,0.000,0,0.000,0,...,1,0,Formulation 4,25.055996,0.964670,0.120510,0.031255,1,2000.0,0
4,52,1.000,0.000,0.00,0,0,0.000,0,0.000,0,...,1,0,Formulation 5,24.776899,0.166927,0.149819,0.051115,1,2000.0,0
5,53,1.000,0.000,0.00,0,0,0.000,0,0.000,0,...,1,0,Formulation 6,24.087662,0.153833,0.099604,0.064277,1,2000.0,0
6,54,0.815,0.000,0.00,0,0,0.185,0,0.000,0,...,2,0,Formulation 7,198.047478,3.789760,0.380302,0.045840,1,1630.0,0
7,55,0.459,0.000,0.00,0,0,0.000,0,0.541,0,...,2,0,Formulation 8,24.932233,1.245743,0.351534,0.091538,1,918.0,0
8,56,1.000,0.000,0.00,0,0,0.000,0,0.000,0,...,1,0,Formulation 9,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
9,57,0.645,0.355,0.00,0,0,0.000,0,0.000,0,...,2,0,Formulation 10,1000.000000,0.000000,1.000000,0.000000,0,0.0,0


In [22]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.166667
1,1,0.024598,0.000563,1.000,0.0,0.127693,0.030838,0.083333
2,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
3,1,0.025056,0.000965,1.000,0.0,0.120510,0.031255,0.083333
4,1,0.024777,0.000167,1.000,0.0,0.149819,0.051115,0.083333
5,1,0.024088,0.000154,1.000,0.0,0.099604,0.064277,0.083333
6,1,0.198047,0.003790,0.815,0.0,0.380302,0.045840,0.166667
7,1,0.024932,0.001246,0.459,0.0,0.351534,0.091538,0.166667
8,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
9,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.166667


In [23]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.166667
1,1,0.024598,0.000563,1.000,0.0,0.127693,0.030838,0.083333
2,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
3,1,0.025056,0.000965,1.000,0.0,0.120510,0.031255,0.083333
4,1,0.024777,0.000167,1.000,0.0,0.149819,0.051115,0.083333
5,1,0.024088,0.000154,1.000,0.0,0.099604,0.064277,0.083333
6,1,0.198047,0.003790,0.815,0.0,0.380302,0.045840,0.166667
7,1,0.024932,0.001246,0.459,0.0,0.351534,0.091538,0.166667
8,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.083333
9,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.166667


In [24]:
labeled_data_path = 'data/labeled_' + str(iteration) + '.xlsx'
labeled_norm_path = 'data/labeled_norm_' + str(iteration) + '.xlsx'


In [25]:
results_final.to_excel(labeled_data_path, index=False)
results_normalized.to_excel(labeled_norm_path, index=False)

# Load the labeled data to update the optimizer

In [26]:
labeled_norm = pd.read_excel(labeled_norm_path)
labeled_norm.head()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,48,0.93,0.0,0.07,0,0,0.0,0,0.0,0,...,0.166667,0,Formulation 1,1.000000,0.000000,1.000000,0.000000,0,0.0,0
1,49,1.00,0.0,0.00,0,0,0.0,0,0.0,0,...,0.083333,0,Formulation 2,0.024598,0.000563,0.127693,0.030838,1,1.0,0
2,50,1.00,0.0,0.00,0,0,0.0,0,0.0,0,...,0.083333,0,Formulation 3,1.000000,0.000000,1.000000,0.000000,0,0.0,0
3,51,1.00,0.0,0.00,0,0,0.0,0,0.0,0,...,0.083333,0,Formulation 4,0.025056,0.000965,0.120510,0.031255,1,1.0,0
4,52,1.00,0.0,0.00,0,0,0.0,0,0.0,0,...,0.083333,0,Formulation 5,0.024777,0.000167,0.149819,0.051115,1,1.0,0


In [27]:
ax_client_design = AxClient.load_from_json_file(ax_client_design_path)


In [28]:
loaded_optimizer_path = "optimizer/optimizer_load_" + str(iteration) + ".json"

In [29]:
loaded_ax_client = sdl.load_labeled_data(ax_client_design, labeled_norm_path)
loaded_ax_client.save_to_json_file(loaded_optimizer_path)

[INFO 08-03 06:46:11] ax.service.ax_client: Completed trial 48 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-03 06:46:11] ax.service.ax_client: Completed trial 49 with data: {'Solu': (1.0, 0), 'Size': (0.024598, 0.000563), 'PDI': (0.127693, 0.030838)}.
[INFO 08-03 06:46:11] ax.service.ax_client: Completed trial 50 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-03 06:46:11] ax.service.ax_client: Completed trial 51 with data: {'Solu': (1.0, 0), 'Size': (0.025056, 0.000965), 'PDI': (0.12051, 0.031255)}.
[INFO 08-03 06:46:11] ax.service.ax_client: Completed trial 52 with data: {'Solu': (1.0, 0), 'Size': (0.024777, 0.000167), 'PDI': (0.149819, 0.051115)}.
[INFO 08-03 06:46:11] ax.service.ax_client: Completed trial 53 with data: {'Solu': (1.0, 0), 'Size': (0.024088, 0.000154), 'PDI': (0.099604, 0.064277)}.
[INFO 08-03 06:46:11] ax.service.ax_client: Completed trial 54 with data: {'Solu': (0.815, 0), 'Size': (0.198047, 0.00379),